In [36]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

In [2]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

  Cloning https://github.com/NVIDIA/apex.git to /tmp/pip-install-3qoyz7j5/apex
  Running setup.py bdist_wheel for fvcore ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/80/eb/49/83b9d20a804f1b4b163d1c1451c670a2067a00175662516f01
  Running setup.py bdist_wheel for apex ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-tovzmrhz/wheels/20/ef/9d/1967e1ee0ae20e7dc8e41ab7208017893b0a026243189508a3
  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built fvcore apex termcolor
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# Need to add this to requirements.txt
!pip install tensorboard

    100% |████████████████████████████████| 3.0MB 11.2MB/s ta 0:00:01
    100% |████████████████████████████████| 19.7MB 2.4MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 29.6MB/s ta 0:00:01
    100% |████████████████████████████████| 112kB 34.6MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 31.1MB/s ta 0:00:01
    100% |████████████████████████████████| 788kB 28.1MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 30.9MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 30.8MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 23.5MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 37.2MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 36.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for grpcio ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/1e/e9/13/2ce6c99171a977bad2f9a4bccfa596a6e7ea060b9fbff51bc3
  Running setup.py bdist_wheel for absl-py ... done
  Stored in d

end basic config, begin training

models to be trained for RA_2_20:

350 augmented, 100 unaugmented
250 augmented, 200 unaugmented
150 augmented, 300 unaugmented

If these look promising, perhaps we break into 50s. This would require changing the yml files in order to have checkpoints to be every 50 instead of every 100 epochs

first pipeline: 350 augmented, 100 unaugmented

In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 32 \
    train.batch_size 128 \
    dataset.name CIFAR10_RA_2_20 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00 \
    scheduler.epochs 350

[2020-07-02 19:25:38] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_2_20
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 32
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
 

In [ ]:
# Errors
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
#!python train.py --config configs/cifar/resnet.yaml \
!python train.py --config /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00350.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume350_100 \
    scheduler.epochs 100

#### Set LEARNING RATE based on ending LR


In [22]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset - without training on unaugmented data
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00350.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/test_results_350_cifar10

[2020-07-03 22:14:37] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00350.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 25.57it/s]
[2020-07-03 22:14:41] __main__ INFO: Elapsed 3.09
[2020-07-03 22:14:41] __main__ INFO: Loss 1.8401 Accuracy 0.7096


In [26]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume350_100/checkpoint_00100.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume350_100/test_results_0100_cifar10

[2020-07-03 22:18:16] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume350_100/checkpoint_00100.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 25.51it/s]
[2020-07-03 22:18:20] __main__ INFO: Elapsed 3.10
[2020-07-03 22:18:20] __main__ INFO: Loss 0.4560 Accuracy 0.8688


In [24]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset - without training on unaugmented data
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00350.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/test_results_0350_cifar101

[2020-07-03 22:14:48] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00350.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 16.76it/s]
[2020-07-03 22:14:49] __main__ INFO: Elapsed 0.96
[2020-07-03 22:14:49] __main__ INFO: Loss 3.2345 Accuracy 0.5380


In [25]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume350_100/checkpoint_00100.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume350_100/test_results_0100_cifar101

[2020-07-03 22:14:52] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume350_100/checkpoint_00100.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 18.55it/s]
[2020-07-03 22:14:54] __main__ INFO: Elapsed 0.86
[2020-07-03 22:14:54] __main__ INFO: Loss 0.8289 Accuracy 0.7760


second pipeline: augmented 250, unaugmented 200


In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 32 \
    train.batch_size 128 \
    dataset.name CIFAR10_RA_2_20 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00 \
    scheduler.epochs 250

In [ ]:
# Errors
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
#!python train.py --config configs/cifar/resnet.yaml \
!python train.py --config /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00250.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume250_200 \
    scheduler.epochs 200

#### Set LEARNING RATE based on ending LR

In [27]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset - without training on unaugmented data
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00250.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/test_results_0250_cifar10

[2020-07-03 22:19:44] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00250.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 25.43it/s]
[2020-07-03 22:19:48] __main__ INFO: Elapsed 3.11
[2020-07-03 22:19:48] __main__ INFO: Loss 1.6258 Accuracy 0.7112


In [28]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume250_200/checkpoint_00200.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume250_200/test_results_0200_cifar10

[2020-07-03 22:19:52] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume250_200/checkpoint_00200.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 25.39it/s]
[2020-07-03 22:19:56] __main__ INFO: Elapsed 3.11
[2020-07-03 22:19:56] __main__ INFO: Loss 0.4352 Accuracy 0.8695


In [29]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset - without training on unaugmented data
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00250.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/test_results_0250_cifar101

[2020-07-03 22:19:59] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00250.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 19.62it/s]
[2020-07-03 22:20:00] __main__ INFO: Elapsed 0.82
[2020-07-03 22:20:00] __main__ INFO: Loss 2.8602 Accuracy 0.5350


In [30]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume250_200/checkpoint_00200.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume250_200/test_results_0200_cifar101

[2020-07-03 22:20:03] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume250_200/checkpoint_00200.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 19.21it/s]
[2020-07-03 22:20:04] __main__ INFO: Elapsed 0.84
[2020-07-03 22:20:04] __main__ INFO: Loss 0.7807 Accuracy 0.7715


third pipeline: randaugment 150, refine 300

In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 32 \
    train.batch_size 128 \
    dataset.name CIFAR10_RA_2_20 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00 \
    scheduler.epochs 150

In [ ]:
# Errors
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
#!python train.py --config configs/cifar/resnet.yaml \
!python train.py --config /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00150.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume150_300 \
    scheduler.epochs 300

#### Set LEARNING RATE based on ending LR

In [31]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset - without training on unaugmented data
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00150.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/test_results_0150_cifar10

[2020-07-03 22:22:21] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00150.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 25.29it/s]
[2020-07-03 22:22:25] __main__ INFO: Elapsed 3.13
[2020-07-03 22:22:25] __main__ INFO: Loss 1.3270 Accuracy 0.7220


In [32]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume150_300/checkpoint_00300.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume150_300/test_results_0300_cifar10

[2020-07-03 22:22:28] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume150_300/checkpoint_00300.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 25.71it/s]
[2020-07-03 22:22:32] __main__ INFO: Elapsed 3.07
[2020-07-03 22:22:32] __main__ INFO: Loss 0.4104 Accuracy 0.8733


In [33]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset - without training on unaugmented data
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00150.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/test_results_0150_cifar101

[2020-07-03 22:22:36] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00/checkpoint_00150.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 19.40it/s]
[2020-07-03 22:22:37] __main__ INFO: Elapsed 0.83
[2020-07-03 22:22:37] __main__ INFO: Loss 2.4433 Accuracy 0.5390


In [34]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume150_300/checkpoint_00300.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume150_300/test_results_0300_cifar101

[2020-07-03 22:22:40] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/exp00_resume150_300/checkpoint_00300.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 19.43it/s]
[2020-07-03 22:22:41] __main__ INFO: Elapsed 0.83
[2020-07-03 22:22:41] __main__ INFO: Loss 0.7315 Accuracy 0.7755


writeout results to s3
*********** MANUAL INTERVENTION NEEDED BELOIW *******************



In [44]:
import pandas as pd
a = pd.Series(['resnet_basic_32_ra_2_20', 350, 'cifar10', 1.8401, 0.7096]) #Loss 0.4279 Accuracy 0.8670
b = pd.Series(['resnet_basic_32_ra_2_20', 350, 'cifar10.1', 3.2345, 0.5380]) #Loss 0.8061 Accuracy 0.7645
c = pd.Series(['resnet_basic_32_ra_2_20_refined350', 100, 'cifar10', 0.3978, 0.8700]) #Loss 0.4342 Accuracy 0.8740
d = pd.Series(['resnet_basic_32_ra_2_20_refined350', 100, 'cifar10.1', 0.8289, 0.7760]) #Loss 0.7919 Accuracy 0.7635


e = pd.Series(['resnet_basic_32_ra_2_20', 250, 'cifar10', 1.6258, 0.7112]) #Loss 0.4279 Accuracy 0.8670
g = pd.Series(['resnet_basic_32_ra_2_20', 250, 'cifar10.1', 2.8602, 0.5350]) #Loss 0.8061 Accuracy 0.7645
f = pd.Series(['resnet_basic_32_ra_2_20_refined250', 200, 'cifar10', 0.4352, 0.8695]) #Loss 0.4342 Accuracy 0.8740
h = pd.Series(['resnet_basic_32_ra_2_20_refined250', 200, 'cifar10.1', 0.7807, 0.7715]) #Loss 0.7919 Accuracy 0.7635

i = pd.Series(['resnet_basic_32_ra_2_20', 150, 'cifar10', 1.3270, 0.7220]) #Loss 0.4279 Accuracy 0.8670
j = pd.Series(['resnet_basic_32_ra_2_20', 150, 'cifar10.1', 2.4433, 0.5390]) #Loss 0.4342 Accuracy 0.8740
k = pd.Series(['resnet_basic_32_ra_2_20_refined150', 300, 'cifar10', 0.4104, 0.8733]) #Loss 0.8061 Accuracy 0.7645
l = pd.Series(['resnet_basic_32_ra_2_20_refined150', 300, 'cifar10.1', 0.7755, 0.7315]) #Loss 0.7919 Accuracy 0.7635
               
df_results = pd.concat([a,b,c,d,e,f, g, h, i, j, k, l], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 92.5 if row[2] == 'cifar10' else 84.9), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (92.0, 93.0) if row[2] == 'cifar10' else (83.2, 86.4)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/july2/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnet_basic_32_ra_2_20,350,cifar10,1.8401,0.7096,92.5,"(92.0, 93.0)"
1,resnet_basic_32_ra_2_20,350,cifar10.1,3.2345,0.538,84.9,"(83.2, 86.4)"
2,resnet_basic_32_ra_2_20_refined350,100,cifar10,0.3978,0.87,92.5,"(92.0, 93.0)"
3,resnet_basic_32_ra_2_20_refined350,100,cifar10.1,0.8289,0.776,84.9,"(83.2, 86.4)"
4,resnet_basic_32_ra_2_20,250,cifar10,1.6258,0.7112,92.5,"(92.0, 93.0)"
5,resnet_basic_32_ra_2_20_refined250,200,cifar10,0.4352,0.8695,92.5,"(92.0, 93.0)"
6,resnet_basic_32_ra_2_20,250,cifar10.1,2.8602,0.535,84.9,"(83.2, 86.4)"
7,resnet_basic_32_ra_2_20_refined250,200,cifar10.1,0.7807,0.7715,84.9,"(83.2, 86.4)"
8,resnet_basic_32_ra_2_20,150,cifar10,1.327,0.722,92.5,"(92.0, 93.0)"
9,resnet_basic_32_ra_2_20,150,cifar10.1,2.4433,0.539,84.9,"(83.2, 86.4)"


In [22]:
#error
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0200_cifar101/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0200_cifar101/predictions.npz'

In [36]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/resnet_basic_32_ra_2_20'
path = '/home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            print("Local File:", os.path.join(path, file))
            print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)

Local File: /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/results.csv
      Dest: sagemaker/results/original-models/resnet_basic_32_ra_2_20/results.csv
Local File: /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/exp00/log_plain.txt
      Dest: sagemaker/results/original-models/resnet_basic_32_ra_2_20/exp00/log_plain.txt
Local File: /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/exp00/checkpoint_00100.pth
      Dest: sagemaker/results/original-models/resnet_basic_32_ra_2_20/exp00/checkpoint_00100.pth
Local File: /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/exp00/checkpoint_00200.pth
      Dest: sagemaker/results/original-models/resnet_basic_32_ra_2_20/exp00/checkpoint_00200.pth
Local File: /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/exp00/env.yaml
      Dest: sagemaker/results/original-models/resnet_basic_32_ra_2_20/exp00/env.yaml
Local File: /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/exp00/log.

In [38]:
import pandas as pd

In [42]:
!pwd

/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass


In [41]:
old_res = pd.read_csv("../../original_model_results/resnet_basic_32_ra_2_20/results.csv")
old_res

,Unnamed: 0,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI,Unnamed: 0.1
0,0,resnet_basic_32_ra_2_20,200,cifar10,1.4212,0.7254,92.5,"(92.0, 93.0)",NaN
1,1,resnet_basic_32_ra_2_20,100,cifar10,1.6066,0.7281,92.5,"(92.0, 93.0)",NaN
2,2,resnet_basic_32_ra_2_20,200,cifar10.1,2.5458,0.5505,84.9,"(83.2, 86.4)",NaN
3,3,resnet_basic_32_ra_2_20,100,cifar10.1,1.9095,0.5490,84.9,"(83.2, 86.4)",NaN
4,4,resnet_basic_32_ra_2_20_refined200,250,cifar10,0.4312,0.8691,92.5,"(92.0, 93.0)",NaN
5,5,resnet_basic_32_ra_2_20_refined100,350,cifar10,0.3978,0.8700,92.5,"(92.0, 93.0)",NaN
6,6,resnet_basic_32_ra_2_20_refined200,250,cifar10.1,0.7625,0.7630,84.9,"(83.2, 86.4)",NaN
7,7,resnet_basic_32_ra_2_20_refined100,350,cifar10.1,0.7138,0.7700,84.9,"(83.2, 86.4)",NaN
8,0,resnet_basic_32_ra_2_20,400,cifar10,1.7503,0.7241,92.5,"(92.0, 93.0)",0.0
9,1,resnet_basic_32_ra_2_20,300,cifar10,1.6118,0.7259,92.5,"(92.0, 93.0)",1.0


In [27]:
temp_df = pd.concat([df_results, old_res], axis = 0)
temp_df

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI,Unnamed: 0
0,resnet_basic_32_ra_2_20,200,cifar10,1.4212,0.7254,92.5,"(92.0, 93.0)",NaN
1,resnet_basic_32_ra_2_20,100,cifar10,1.6066,0.7281,92.5,"(92.0, 93.0)",NaN
2,resnet_basic_32_ra_2_20,200,cifar10.1,2.5458,0.5505,84.9,"(83.2, 86.4)",NaN
3,resnet_basic_32_ra_2_20,100,cifar10.1,1.9095,0.549,84.9,"(83.2, 86.4)",NaN
4,resnet_basic_32_ra_2_20_refined200,250,cifar10,0.4312,0.8691,92.5,"(92.0, 93.0)",NaN
5,resnet_basic_32_ra_2_20_refined100,350,cifar10,0.3978,0.87,92.5,"(92.0, 93.0)",NaN
6,resnet_basic_32_ra_2_20_refined200,250,cifar10.1,0.7625,0.763,84.9,"(83.2, 86.4)",NaN
7,resnet_basic_32_ra_2_20_refined100,350,cifar10.1,0.7138,0.77,84.9,"(83.2, 86.4)",NaN
0,resnet_basic_32_ra_2_20,400,cifar10,1.7503,0.7241,92.5,"(92.0, 93.0)",0.0
1,resnet_basic_32_ra_2_20,300,cifar10,1.6118,0.7259,92.5,"(92.0, 93.0)",1.0


In [31]:
temp_df.to_csv("../../original_model_results/resnet_basic_32_ra_2_20/results.csv")

In [30]:
pd.read_csv("../../original_model_results/resnet_basic_32_ra_2_20/results.csv")

,Unnamed: 0,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI,Unnamed: 0.1
0,0,resnet_basic_32_ra_2_20,200,cifar10,1.4212,0.7254,92.5,"(92.0, 93.0)",NaN
1,1,resnet_basic_32_ra_2_20,100,cifar10,1.6066,0.7281,92.5,"(92.0, 93.0)",NaN
2,2,resnet_basic_32_ra_2_20,200,cifar10.1,2.5458,0.5505,84.9,"(83.2, 86.4)",NaN
3,3,resnet_basic_32_ra_2_20,100,cifar10.1,1.9095,0.5490,84.9,"(83.2, 86.4)",NaN
4,4,resnet_basic_32_ra_2_20_refined200,250,cifar10,0.4312,0.8691,92.5,"(92.0, 93.0)",NaN
5,5,resnet_basic_32_ra_2_20_refined100,350,cifar10,0.3978,0.8700,92.5,"(92.0, 93.0)",NaN
6,6,resnet_basic_32_ra_2_20_refined200,250,cifar10.1,0.7625,0.7630,84.9,"(83.2, 86.4)",NaN
7,7,resnet_basic_32_ra_2_20_refined100,350,cifar10.1,0.7138,0.7700,84.9,"(83.2, 86.4)",NaN
8,0,resnet_basic_32_ra_2_20,400,cifar10,1.7503,0.7241,92.5,"(92.0, 93.0)",0.0
9,1,resnet_basic_32_ra_2_20,300,cifar10,1.6118,0.7259,92.5,"(92.0, 93.0)",1.0
